This notebook reads our DC2 synth model and saves the CSAMPLES file to 

/e/ocean1/users/vargatn/LSST/SYNTHETIC

In [1]:
from importlib import reload
import fitsio as fio
import numpy as np
import pandas as pd

import healpy as hp
import copy
import sys
import glob
import os
import matplotlib.pyplot as plt
%matplotlib inline

import sklearn.decomposition as decomp


import matplotlib as mpl
import subprocess as sp
import scipy.interpolate as interpolate
import pickle as pickle

import multiprocessing as mp

import synthetic.tools as tools
import synthetic.emulator.emulator as emulator
import synthetic.emulator.indexer as indexer
import synthetic.emulator.reader as reader

In [2]:
tag_root = "dc2-alpha_concentric_sample-v01_test-03"
NREPEATS = 4
NSAMPLES = 1600000
NCHUNKS = 160
bandwidth=0.1

root_path = "/e/ocean1/users/vargatn/LSST/DC2_1.1.4/clusters_v01/resamples/"
deep_data_path = "/e/ocean1/users/vargatn/LSST/DC2_1.1.4/clusters_v01/dc2_cluster_sim_cutouts/cosmoDC2_v1.1.4_refpixels.h5"
wide_data_path = "/e/ocean1/users/vargatn/LSST/DC2_1.1.4/clusters_v01/dc2_cluster_sim_cutouts/clust_dc2-sim-LOS_v1.h5"

LOGR_DRAW_RMINS = np.array([-3, -0.5, 0., 0.5])
LOGR_DRAW_RMAXS = np.array([-0.5, 0., 0.5, 1.2])
LOGR_CAT_RMAXS = [0., 0.5, 1.1, 1.2]

deep_smc_settings = {
    "columns": [
        ("GABS", ("ellipticity_1_true", "ellipticity_2_true", "SQSUM")),
        ("SIZE", "size_true"),
        ("MAG_I", "mag_i"),
        ("COLOR_G_R", ("mag_g", "mag_r", "-")),
        ("COLOR_R_I", ("mag_r", "mag_i", "-")),
        ("COLOR_I_Z", ("mag_i", "mag_z", "-")),
        ("STELLAR_MASS", "stellar_mass"),
        ("HALO_MASS", "halo_mass")        
    ],
    "logs": [False, True, False, False, False, False, True, True],
    "limits": [(0., 1.), (-1, 5), (17, 25), (-1, 3), (-1, 3), (-1, 3), (10**3, 10**13), (10**9, 10**16)],
}

wide_cr_settings = {
    "columns": [
        ("GABS", ("ellipticity_1_true", "ellipticity_2_true", "SQSUM")),
        ("SIZE", "size_true"),
        ("MAG_I", "mag_i"),
        ("COLOR_G_R", ("mag_g", "mag_r", "-")),
        ("COLOR_R_I", ("mag_r", "mag_i", "-")),
        ("COLOR_I_Z", ("mag_i", "mag_z", "-")),
        ("STELLAR_MASS", "stellar_mass"),
        ("HALO_MASS", "halo_mass"),
        ("LOGR", "R"),        
    ],
    "logs": [False, True, False, False, False, False, True, True, True],
    "limits": [(0., 1.), (-1, 5), (17, 25), (-1, 3), (-1, 3), (-1, 3), (10**3, 10**13), (10**9, 10**16), (1e-3, 16)],
}

columns = {
    "cols_dc": ["COLOR_G_R", "COLOR_R_I",],
    "cols_wr": ["LOGR",],
    "cols_wcr": ["COLOR_G_R", "COLOR_R_I", "LOGR",],
}


In [3]:
deep_data_path = "/e/ocean1/users/vargatn/LSST/DC2_1.1.4/clusters_v01/dc2_cluster_sim_cutouts/cosmoDC2_v1.1.4_refpixels.h5"
wide_data_path = "/e/ocean1/users/vargatn/LSST/DC2_1.1.4/clusters_v01/dc2_cluster_sim_cutouts/clust_dc2-sim-LOS_v1.h5"

In [4]:
refpixel = pd.read_hdf(deep_data_path, key="data")
table = pd.read_hdf(wide_data_path, key="data")

In [5]:
tmp_wide_cr_settings = wide_cr_settings.copy()
# tmp_wide_cr_settings["limits"][-1] = (10**-3, 10**LOGR_CAT_RMAXS[i])
_wide_cr_settings_rands = emulator.construct_deep_container(refpixel, tmp_wide_cr_settings)

# loading deep catalogs
_deep_smc_settings = emulator.construct_deep_container(refpixel, deep_smc_settings)

# loading cluster data
tmp_wide_cr_settings = wide_cr_settings.copy()
# tmp_wide_cr_settings["limits"][-1] = (10**-3, 10**LOGR_CAT_RMAXS[i])
_wide_cr_settings_clust = emulator.construct_deep_container(table, tmp_wide_cr_settings)


(1513572, 9)
(1513572,)
(1513572, 8)
(1513572,)
(1009658, 9)
(1009658,)


In [6]:
samples = []
scores = []
for rbin in np.arange(4):
    print(rbin)
#     expr = "/e/ocean1/users/vargatn/EMULATOR/EPSILON/resamples/epsilon_concentric_sample_v06_run*_rbin" + str(rbin) + "*samples.fits" 
    expr = "/e/ocean1/users/vargatn/LSST/DC2_1.1.4/clusters_v01/resamples/dc2-alpha_concentric_sample-v01_test-03/dc2-alpha_concentric_sample-v01_test-03_run0*_rbin" + str(rbin) + "*samples.fits" 

    fnames_samples = np.sort(glob.glob(expr))
    fnames_scores = []
    for fname in fnames_samples:
        fnames_scores.append(fname.replace("samples.fits", "scores.fits"))

    samples_sep = []
    scores_sep = []
    for i, fname in enumerate(fnames_samples):
#         print(fname)
        samples_sep.append(fio.read(fname))
        scores_sep.append(fio.read(fnames_scores[i]))
        
    samples.append(np.hstack(samples_sep))
    scores.append(np.hstack(scores_sep))

0
1
2
3


In [7]:


mag_lims = (17, 22.5)
r_lims_all = [(-1.5, -0.5), (-0.5, 0.), (0, 0.5), (0.5, 1.0)]
redges = [-1.5, -0.5, 0., 0.5, 1.0]
rareas = np.array([np.pi*((10**redges[i+1])**2. - (10**redges[i])**2.) for i in np.arange(len(redges)-1)])



In [8]:


magcol = "mag_i"
ii = ((table[magcol] > mag_lims[0]) & (table[magcol] < mag_lims[1]))

clust_los_nums = np.histogram(np.log10(table[ii]["R"]), bins=redges)[0] / 41 # / nc

ii = ((refpixel[magcol] > mag_lims[0]) & (refpixel[magcol] < mag_lims[1]))
surfdens = len(refpixel[ii]) / hp.nside2pixarea(32, degrees=True) / 3600 / 3
rands_los_nums = surfdens * rareas
# rands_los_nums = np.histogram(np.log10(refpixel[ii]["R"]), bins=redges)[0] / rareas#* ratio# / nr
nratios = clust_los_nums / rands_los_nums



/home/moon/vargatn/anaconda3/envs/galsim/lib/python3.7/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [9]:


ifields2 = []
iclusts2 = []
i2ds2 = []
for rbin in np.arange(4):
    print(rbin)
    _ifield, _iclust, _i2d = reader.result_reader2(samples[rbin], scores[rbin], nratio=nratios[rbin], m_factor=100, seed=rbin)
    ifields2.append(_ifield)
    iclusts2.append(_iclust)
    i2ds2.append(_i2d)



0
2.741989144315195
1
1.7913041757345514
2
1.1243442810539395
3
0.793271619957704


 In this exercise we only draw cluster samples, as the clusters will be injected to DC2

In [10]:


csamples = []
for rbin in np.arange(4):
    print(rbin)
    tab = pd.DataFrame.from_records(samples[rbin][iclusts2[rbin]].byteswap().newbyteorder())
    tab.drop('index', axis=1, inplace=True)
    kde = emulator.KDEContainer(tab)
    kde.standardize_data()
    kde.construct_kde(0.1)
    _csample = kde.random_draw(4e6)
    csamples.append(_csample)



0
1
2
3


In [11]:
csamples

[             GABS      SIZE      MAG_I  COLOR_G_R  COLOR_R_I  COLOR_I_Z  \
 0        0.577953 -0.609898  24.705472  -0.441677   0.149741   0.150836   
 1        0.012229 -0.484255  24.692372   0.366218   0.190196   0.083731   
 2        0.101226 -0.354478  23.370602   0.128018   0.597502   0.376897   
 3        0.056248 -0.449896  22.256638   1.275530   0.541578   0.212120   
 4        0.097097 -0.590536  24.942514   0.342835   0.064479   0.235198   
 ...           ...       ...        ...        ...        ...        ...   
 3999995  0.413577 -0.253723  23.680008   1.227264   0.527737   0.239330   
 3999996  0.556364 -0.909532  23.424839   1.226261   0.293748   0.135260   
 3999997  0.111252 -0.560508  24.640485   1.351195   0.567157   0.192104   
 3999998  0.208725 -0.837587  25.069540   0.145456   0.614259   0.202327   
 3999999  0.079152 -1.041265  23.668925   1.037197   0.369383   0.194410   
 
          STELLAR_MASS  HALO_MASS      LOGR  
 0            9.918076  11.930301 -1.325

In [12]:
opath = "/e/ocean1/users/vargatn/LSST/SYNTHETIC/csamples_nov_dev_01.h5"

In [13]:
reload(tools)

<module 'synthetic.tools' from '/home/moon/vargatn/DES/SYNTHETIC/synthetic/synthetic/tools.py'>

In [14]:
tools.save(opath, csamples)

In [16]:
ctab = _wide_cr_settings_clust["container"].data
rtab = _wide_cr_settings_rands["container"].data

In [17]:
opath = "/e/ocean1/users/vargatn/LSST/SYNTHETIC/csamples_nov_dev_01_ctab.h5"
ctab.to_hdf(opath, key="data")
opath = "/e/ocean1/users/vargatn/LSST/SYNTHETIC/csamples_nov_dev_01_rtab.h5"
rtab.to_hdf(opath, key="data")

In [18]:
ctab

,GABS,SIZE,MAG_I,COLOR_G_R,COLOR_R_I,COLOR_I_Z,STELLAR_MASS,HALO_MASS,LOGR
0,0.104769,-0.908358,24.833206,-0.034159,0.406742,0.162960,8.071676,11.058885,1.095340
1,0.343897,-0.720387,24.877115,0.593456,0.701340,0.691994,8.784044,11.290493,1.049529
2,0.024317,-0.979597,24.186816,1.238859,0.669086,0.174717,8.323544,10.777160,1.122350
3,0.067596,-0.779295,24.624785,1.165337,0.829702,0.195393,8.209156,11.588618,1.124467
4,0.089344,-0.654554,24.328617,0.996740,0.657883,0.102413,8.361197,10.959500,1.128371
...,...,...,...,...,...,...,...,...,...
1009653,0.346643,-0.525755,22.836572,0.535080,0.873455,0.438391,9.738565,12.758043,1.103064
1009654,0.025276,-0.836426,24.841200,0.606043,0.938686,0.381571,8.156223,10.757855,1.185429
1009655,0.550381,-1.082066,24.534380,0.827349,1.085112,0.370970,8.578614,10.830405,1.100807
1009656,0.530662,-0.789418,24.572302,0.227751,0.236155,0.612955,9.103561,11.687274,0.566554
